In [1]:
import polars as pl

In [2]:
# data_path = './data/data_1mb.csv'
# data_path = './data/data_1gb.csv'
# data_path = './data/data_10gb.csv'
data_path = './data/data_30gb.csv'

In [3]:
df = pl.read_csv(data_path)

In [5]:
df.height

306783378

In [4]:
df.head()

user_id,timestamp,amount,currency,card_number,merchant,country,is_fraud
i64,str,f64,str,i64,str,str,bool
892,"""2025-01-30T06:46:59.148540""",1693.53,"""USD""",4800977575264,"""Nichols LLC""","""Singapore""",false
456,"""2025-04-20T11:55:18.554499""",2426.07,"""USD""",4384766118210420,"""Greene Inc""","""Chile""",false
141,"""2025-03-07T14:24:03.622931""",3342.85,"""USD""",213144278641609,"""Johnson Group""","""Aruba""",false
746,"""2025-02-28T17:11:27.285112""",2342.18,"""USD""",213164815767014,"""Werner, Gonzales and Phillips""","""New Caledonia""",false
613,"""2025-02-18T22:04:18.463873""",2791.12,"""USD""",30013009327088,"""Petty-Mcclain""","""Seychelles""",false


In [ ]:
df.describe()

In [39]:
# ile unikalnych userów?
len(df['user_id'].unique())



1000

In [40]:
# popularnosc sprzedawcow
df.group_by("merchant").count().sort("count", descending=True)

/var/folders/xk/bnpzxy5j2t336cl3jrv0gp0h0000gn/T/ipykernel_50146/353970258.py:2: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  df.group_by("merchant").count().sort("count", descending=True)


merchant,count
str,u32
"""Williams Ltd""",172212
"""Smith Group""",165563
"""Smith PLC""",164450
"""Smith LLC""",164037
"""Smith Ltd""",141550
…,…
"""Parker, Nunez and Thomas""",1
"""Brennan, Williams and Sampson""",1
"""Rush, Martinez and Flores""",1


In [41]:
# fraud detection

# większość transakcji jednego usera jest z jednego kraju, 
# jako fraud bedziemy identyfikowali transakcje ktore są wykonane z innego kraju

# Obliczamy liczbę transakcji dla każdej pary user-kraj
user_country_counts = (
    df.group_by(['user_id', 'country'])
    .len()
    .sort(['user_id', 'len'], descending=[False, True])
)

# Dla każdego usera bierzemy kraj z największą liczbą transakcji
user_most_common_country = (
    user_country_counts
    .group_by('user_id')
    .first()
    .select(['user_id', 'country'])
    .rename({'country': 'country_most_common'})
)

# Dodajemy kolumnę z najczęstszym krajem dla danego usera
df_with_common_country = df.join(
    user_most_common_country,
    on='user_id'
)

# Flagujemy transakcje które są wykonane z innego kraju
df_with_anomaly = df_with_common_country.with_columns(
    (pl.col('country') != pl.col('country_most_common'))
    .alias('is_country_anomaly')
)

# Show transactions that are potential fraud based on country anomaly
country_anomaly_transactions = df_with_anomaly.filter(pl.col('is_country_anomaly'))

In [42]:
country_anomaly_transactions

user_id,timestamp,amount,currency,card_number,merchant,country,is_fraud,country_most_common,is_country_anomaly
i64,str,f64,str,i64,str,str,bool,str,bool
276,"""2025-02-26T04:12:06.093832""",4267.2,"""USD""",379654095189236,"""Shaw, Gross and Smith""","""Hungary""",true,"""Burundi""",true
997,"""2025-01-18T19:49:20.107546""",1235.01,"""USD""",213101170861446,"""Gray, Lopez and Diaz""","""China""",true,"""Canada""",true
845,"""2025-05-05T14:59:25.889815""",436.8,"""USD""",639017064739,"""Chavez, Smith and Whitney""","""Australia""",true,"""Japan""",true
61,"""2025-01-26T05:27:08.790318""",3358.25,"""USD""",4389224905053061,"""Lowery-Wilson""","""Western Sahara""",true,"""Guyana""",true
109,"""2025-01-29T10:36:37.154047""",3371.63,"""USD""",4501378789757,"""Lopez-Christensen""","""Czech Republic""",true,"""Slovakia (Slovak Republic)""",true
…,…,…,…,…,…,…,…,…,…
785,"""2025-04-12T16:12:42.929146""",1057.88,"""USD""",213155284249245,"""Eaton, Harper and Bennett""","""Belize""",true,"""Cameroon""",true
29,"""2025-01-03T15:56:27.345241""",1914.58,"""USD""",213144784346107,"""Stephens-Gibson""","""Algeria""",true,"""Singapore""",true
229,"""2025-03-16T10:28:06.480631""",2470.91,"""USD""",3596037048821272,"""Snow-Ortiz""","""Oman""",true,"""Libyan Arab Jamahiriya""",true


In [43]:
len(country_anomaly_transactions) / len(df) * 100.0

0.9975204307428924